# Neptune Connection Testing

This notebook tests connection to Neptune using neptune-python-utils.

In [ ]:
# Install neptune-python-utils if not already installed
!pip install neptune-python-utils

In [ ]:
import os
import boto3
import logging
from neptune_python_utils.endpoints import Endpoints
from neptune_python_utils.gremlin_utils import GremlinUtils

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Neptune endpoint
CLUSTER_NAME = "graph-rag-originofcovid19dataset-benchmark"
ENDPOINT = f"{CLUSTER_NAME}.cluster-c7m8ay28gj4o.us-west-2.neptune.amazonaws.com"
PORT = 8182

# Create endpoints with proxy support
endpoints = Endpoints(
    neptune_endpoint=ENDPOINT,
    region_name='us-west-2',
    proxy_dns='localhost',  # Using localhost since we're in a notebook
    proxy_port=PORT,
    remove_host_header=False  # Important for ALB
)

print(f"Database URL: wss://{ENDPOINT}:{PORT}/gremlin")

In [ ]:
# Initialize Gremlin utilities
GremlinUtils.init_statics(globals())
gremlin_utils = GremlinUtils(endpoints)

try:
    # Create connection with SSL verification disabled since we're using a proxy
    print("\nInitializing connection...")
    conn = gremlin_utils.remote_connection(ssl=False)
    
    # Create traversal source
    print("Creating traversal source...")
    g = gremlin_utils.traversal_source(connection=conn)
    
    # Test simple query
    print("\nTesting query...")
    result = g.inject(1).toList()  # Simple test query that doesn't require graph access
    print(f"Query result: {result}")
    
    print("\nConnection test successful!")
    
except Exception as e:
    print(f"\nConnection failed: {str(e)}")
    raise
    
finally:
    # Clean up
    if 'conn' in locals():
        print("\nClosing connection...")
        conn.close()

In [ ]:
# Test a more complex query
try:
    # Create a new connection
    conn = gremlin_utils.remote_connection(ssl=False)
    g = gremlin_utils.traversal_source(connection=conn)
    
    # Try to get vertex count
    print("\nGetting vertex count...")
    count = g.V().count().next()
    print(f"Number of vertices: {count}")
    
    # Try to get some vertex properties
    print("\nGetting vertex properties...")
    vertices = g.V().limit(5).valueMap(True).toList()
    print("Sample vertices:")
    for v in vertices:
        print(f"\n{v}")
        
except Exception as e:
    print(f"\nQuery failed: {str(e)}")
    raise
    
finally:
    if 'conn' in locals():
        conn.close()